In [2]:
import pandas as pd

df = pd.read_parquet("../data/processed/RM_Dimessi_clean.parquet")

In [3]:
# Visualizza tutte le colonne disponibili
print(df.columns.tolist())

['anno', 'età del paziente dimesso', 'sesso', 'comnas', 'comres', 'cirrom', 'cittad', 'staciv', 'titstu', 'profes', 'profes1', 'profes2', 'data_decesso', 'data_inser', 'id', 'data_ins', 'anni_paz', 'codirc', 'polo', 'specrep', 'reparto', 'proric', 'datric', 'regime', 'onere', 'tip_attestato', 'cciec', 'ist_competente', 'cod_qual_ben', 'cod_attestato', 'data_fine_valid', 'tessera_team', 'identificativo_pers', 'prov', 'ircpro', 'reppro', 'proricpro', 'regimepro', 'modacc', 'tempo', 'patriab', 'diapri', 'diase1', 'diase2', 'diase3', 'diase4', 'diase5', 'diase6', 'diase7', 'diase8', 'cognric', 'compric', 'comlingric', 'sensric', 'manipric', 'equilric', 'locomric', 'cardiovric', 'respirric', 'cuteric', 'contrsfinric', 'catvescric', 'nutrric', 'trachric', 'postcomric', 'mieloric', 'bartric1', 'bartric2', 'bartric3', 'bartric4', 'bartric5', 'bartric6', 'bartric7', 'bartric8', 'bartric9', 'bartric10', 'bartric11', 'mwt_metri', 'mwt_borg', 'gosric', 'lcfric', 'totale_drsric', 'GCA', 'GCS', 'DCA

In [4]:
df["datric"] = pd.to_datetime(df["datric"], errors="coerce")
df["datdim"] = pd.to_datetime(df["datdim"], errors="coerce")

df = df.sort_values(by=["patient_id", "datric"])

df["datdim_prev"] = df.groupby("patient_id")["datdim"].shift(1)
df["intervallo_riammissione"] = (df["datric"] - df["datdim_prev"]).dt.days

df["riammissione_30g"] = df["intervallo_riammissione"].apply(lambda x: 1 if pd.notnull(x) and 0 < x <= 30 else 0)
df["riammissione_90g"] = df["intervallo_riammissione"].apply(lambda x: 1 if pd.notnull(x) and 0 < x <= 90 else 0)

# Verifica
print(df["riammissione_30g"].value_counts(normalize=True))

riammissione_30g
0    1.0
Name: proportion, dtype: float64


In [5]:
df["ricoveri_per_paziente"] = df.groupby("patient_id")["patient_id"].transform("count")
df["ricoveri_per_paziente"].value_counts().sort_index()

ricoveri_per_paziente
1    21141
Name: count, dtype: int64

In [6]:
print(df["datric"].isna().mean(), "percentuale di NaT in datric")
print(df["datdim"].isna().mean(), "percentuale di NaT in datdim")

4.7301452154581144e-05 percentuale di NaT in datric
4.7301452154581144e-05 percentuale di NaT in datdim


# NON CI SONO RIAMMISSIONI